<a href="https://colab.research.google.com/github/bbamknp2018/MSC_DPDM_2025/blob/main/midterm2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ให้สืบหาฆาตรกร (10 คะแนน) และ ผู้บงการ (5 คะแนน) ในคดี ฆาตรกรรม (murder) ที่เกิดขึ้นในวันที่ 15 มกราคม 2018 ที่เมือง Pandas City โดยเริ่มจากการตรวจสอบ ข้อมูลคดีในไฟล์ crime_scene_report.csv

In [5]:
import pandas as pd
csr = pd.read_csv('/content/crime_scene_report.csv')
csr

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,Pandas City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,Pandas City
...,...,...,...,...
1223,20180430,bribery,\n,Garden Grove
1224,20180430,fraud,‘Why not?’ said the March Hare.\n,Houma
1225,20180430,assault,\n,Fontana
1226,20180501,assault,be NO mistake about it: it was neither more no...,Trenton


In [8]:
csr.isnull().any()

,0
date,False
type,False
description,False
city,False


In [9]:
csr.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1228 non-null   int64 
 1   type         1228 non-null   object
 2   description  1228 non-null   object
 3   city         1228 non-null   object
dtypes: int64(1), object(3)
memory usage: 38.5+ KB


In [10]:
csr['type'].unique()


array(['robbery', 'murder', 'theft', 'fraud', 'arson', 'bribery',
       'assault', 'smuggling', 'blackmail'], dtype=object)

In [13]:
print(f"data start date: {csr['date'].min()}")
print(f"data end date: {csr['date'].max()}")

data start date: 20170101
data end date: 20180501


In [14]:
csr.groupby('type').count()


,date,description,city
type,,,
arson,148,148,148
assault,145,145,145
blackmail,130,130,130
bribery,135,135,135
fraud,130,130,130
murder,148,148,148
robbery,134,134,134
smuggling,117,117,117
theft,141,141,141


In [18]:
city_p = csr[csr['city'].str.startswith(('P', 'p'))]
city_p

,date,type,description,city
3,20180215,murder,REDACTED REDACTED REDACTED,Pandas City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,Pandas City
7,20170712,theft,"A lone hunter stalks the night, firing arrows ...",Pandas City
8,20170820,arson,"Wield the Hammer of Sol with honor, Titan, it ...",Pandas City
9,20171110,robbery,The Gjallarhorn shoulder-mounted rocket system...,Pandas City
...,...,...,...,...
1189,20180419,assault,"out here? Run home this moment, and fetch me a...",Pittsburgh
1190,20180419,bribery,‘How do you know I’m mad?’ said Alice.\n,Poughkeepsie
1193,20180420,assault,"So she swallowed one of the cakes, and was del...",Panama City
1198,20180422,assault,"‘Exactly so,’ said Alice.\n",Port Arthur


In [20]:
person= pd.read_csv('/content/person.csv')
inter = pd.read_csv('/content/interview.csv')
dl    = pd.read_csv('/content/drivers_license.csv')
gfm   = pd.read_csv('/content/get_fit_now_member.csv')
gfc   = pd.read_csv('/content/get_fit_now_check_in.csv')
fb    = pd.read_csv('/content/facebook_event_checkin.csv')
inc   = pd.read_csv('/content/income.csv')

# แปลงวันที่ให้เป็น string แบบปลอดภัย
for c in ['date','check_in_date','check_in_time','check_out_time','timestamp']:
    if c in csr.columns:   csr[c]   = csr[c].astype(str)
    if c in gfc.columns:   gfc[c]   = gfc[c].astype(str)
    if c in fb.columns:    fb[c]    = fb[c].astype(str)


#step 1 : หาคดีเป้าหมายจาก crime_scene_report.csv
ที่เกิดขึ้นในวันที่ 15 มกราคม 2018 ที่เมือง Pandas City

In [21]:
case = (csr
        .assign(date=csr['date'].astype(str))
        .query("date == '20180115' and type.str.lower()=='murder'", engine='python')
        .loc[lambda d: d['city'].str.contains('Pandas City', case=False, na=False)]
        .sort_index())

case[['date','type','city','description']]

,date,type,city,description
1227,20180115,murder,Pandas City,Security footage shows that there were 2 witne...


จาก description ของคดี 2018-01-15 (Pandas City) บอกว่า:
มีพยาน 2 คน → คนแรกอยู่บ้านหลังสุดท้ายบน Northwestern Dr
                      คนที่สองชื่อ Annabel อยู่แถว Franklin Ave

#Step 2 หาพยานจาก person.csv
จาก Description ใน step ที่ 1 เราทราบพยาน 2 คน จากนั้นเราจะค้นหาพยานเพื่อสืบสวนต่อไป

In [22]:
# พยานคนแรก: last house on Northwestern Dr → บ้านเลขที่สูงสุดบนถนนนี้
north_witness = (person[person['address_street_name']=='Northwestern Dr']
                 .sort_values('address_number', ascending=False)
                 .head(1))

# พยานคนที่สอง: Annabel บน Franklin Ave
anna_witness = person[(person['name'].str.contains('Annabel', case=False, na=False)) &
                      (person['address_street_name']=='Franklin Ave')]

north_witness, anna_witness

(        id            name  license_id  address_number address_street_name  \
 499  14887  Morty Schapiro      118009            4919     Northwestern Dr   
 
            ssn  
 499  111564949  ,
         id            name  license_id  address_number address_street_name  \
 665  16371  Annabel Miller      490173             103        Franklin Ave   
 
            ssn  
 665  318771143  )

In [23]:
witness_ids = pd.concat([north_witness, anna_witness])['id'].tolist()
witness_testimony = inter[inter['person_id'].isin(witness_ids)]
witness_testimony[['person_id','transcript']]

,person_id,transcript
4988,14887,I heard a gunshot and then saw a man run out. ...
4989,16371,"I saw the murder happen, and I recognized the ..."


**พยานคนที่ 1** เล่าว่า ฉันได้ยินเสียงปืน แล้วเห็นผู้ชายคนหนึ่งวิ่งออกมา เขามีกระเป๋า ‘Get Fit Now Gym’
หมายเลขสมาชิกบนกระเป๋าเริ่มต้นด้วย ‘48Z’ มีแต่สมาชิก Gold เท่านั้น
รถที่เขาขึ้นมีป้ายทะเบียนที่มี ‘H42W’ รวมอยู่

**เบาะแส**

*  ฆาตกรเป็นสมาชิกยิม Get Fit Now
* Membership ID เริ่มด้วย “48Z”
* สถานะสมาชิก = Gold
* รถมีป้ายทะเบียนที่มี “H42W”



**พยานคนที่ 2** เล่าว่า ฉันเห็นการฆาตกรรมเกิดขึ้น และจำได้ว่าฆาตกรคือคนที่ฉันเคยเห็นในยิมเมื่อสัปดาห์ก่อน (9 มกราคม)

**เบาะแส**

*  ยืนยันว่าเป็น สมาชิกยิมเดียวกัน (Get Fit Now)
* วันที่ 9 มกราคม (ก่อนวันฆาตกรรมไม่กี่วัน) เขาอยู่ที่ยิม

#Step 3.1 — หา “สมาชิกยิม” ตามเบาะแส (Gold + ID เริ่มด้วย 48Z)

In [24]:
# 1) สมาชิกที่เป็น Gold และ id เริ่มด้วย "48Z"
m48z_gold = gfm[
    gfm['id'].astype(str).str.startswith('48Z') &
    gfm['membership_status'].str.contains('gold', case=False, na=False)
]
m48z_gold

,id,person_id,name,membership_start_date,membership_status
181,48Z7A,28819,Joe Germuska,20160305,gold
182,48Z55,67318,Jeremy Bowers,20160101,gold


In [34]:
# 1) สมาชิกที่เป็น Gold และ id เริ่มด้วย "48Z"
m48z_gold = gfm[
    gfm['id'].astype(str).str.startswith('48Z') &
    gfm['membership_status'].str.contains('gold', case=False, na=False)
]
print("48Z Gold Member IDs:", m48z_gold['id'].tolist()) # Print the member IDs

# 2) คนในกลุ่มนี้ที่เช็คอินวันที่ 2018-01-09
print("GFC check_in_date values sample:", gfc['check_in_date'].unique()[:10]) # Print sample check-in dates
c9 = gfc[
    (gfc['membership_id'].isin(m48z_gold['id'])) &
    (gfc['check_in_date'].astype(str) == '2018-01-09')
]

# 3) ผูกกลับไปยัง person (เพื่อได้ชื่อคน)
suspects_gym = (c9
    .merge(gfm, left_on='membership_id', right_on='id', how='left', suffixes=('','_mem'))
    .merge(person, left_on='person_id', right_on='id', how='left', suffixes=('','_person'))
    .drop_duplicates(subset=['person_id'])
)[['person_id','name','membership_id','check_in_time','check_out_time']]
suspects_gym

48Z Gold Member IDs: ['48Z7A', '48Z55']
GFC check_in_date values sample: ['20180212' '20170811' '20180429' '20180128' '20171027' '20171018'
 '20171127' '20170627' '20180114' '20171107']


,person_id,name,membership_id,check_in_time,check_out_time


#Step 3.2 — เช็คอินที่ยิมวันที่ 2018-01-09

In [35]:
# คนในกลุ่ม 48Z + gold ที่ไปยิม 'วันเกิดเหตุ'
c15 = gfc[
    gfc['membership_id'].isin(m48z_gold['id']) &
    (gfc['check_in_date'].astype(str) == '2018-01-15')
]

merged_suspects_15 = (c15
    .merge(gfm, left_on='membership_id', right_on='id', how='left')
    .merge(person, left_on='person_id', right_on='id', how='left')
)

print(merged_suspects_15.columns) # Add this line to print columns

suspects_15 = merged_suspects_15[['person_id','name_y','membership_id','check_in_time','check_out_time']]

suspects_15

Index(['membership_id', 'check_in_date', 'check_in_time', 'check_out_time',
       'id_x', 'person_id', 'name_x', 'membership_start_date',
       'membership_status', 'id_y', 'name_y', 'license_id', 'address_number',
       'address_street_name', 'ssn'],
      dtype='object')


,person_id,name_y,membership_id,check_in_time,check_out_time


In [38]:
# ผู้สมัคร 48Z + gold (คุณมีแล้วเป็น 2 คน: Joe Germuska, Jeremy Bowers)
m48z_gold = gfm[gfm['id'].astype(str).str.startswith('48Z') &
                gfm['membership_status'].str.contains('gold', case=False, na=False)]

# ผูกกับ person แล้วต่อด้วยใบขับขี่
cand = (m48z_gold
        .merge(person, left_on='person_id', right_on='id', how='left')
        .merge(dl, left_on='license_id', right_on='id', how='left', suffixes=('','_dl')))

# ดูข้อมูลเบื้องต้นของผู้สมัครทั้งคู่
cand[['name_y','id_x','membership_status','plate_number','car_make','car_model','hair_color','height']]

,name_y,id_x,membership_status,plate_number,car_make,car_model,hair_color,height
0,Joe Germuska,48Z7A,gold,NaN,NaN,NaN,NaN,NaN
1,Jeremy Bowers,48Z55,gold,0H42W2,Chevrolet,Spark LS,brown,70.0


#Step C ผู้ต้องสงสัยจาก “ทะเบียนรถมี H42W” จากใบขับขี่

In [44]:
# ตามจากทะเบียนรถ ว่าคนไหนมีป้ายทะเบียนรถตามเบาะแสจากพยานคนที่ 1
killer_rows = cand[cand['plate_number'].astype(str).str.contains('H42W', na=False)]
killer_rows[['name_y','plate_number','car_make','car_model','hair_color','height']]
killer_name = killer_rows['name_y'].iloc[0]
killer_name

'Jeremy Bowers'

#**[ได้ชื่อฆาตกรชื่อว่า Jeremy Bowers](https://)**

In [46]:
# อ่านคำให้การของฆาตกร
killer_id = int(person.loc[person['name']==killer_name, 'id'].iloc[0])
killer_iv = inter[inter['person_id']==killer_id]
killer_iv[['person_id','transcript']]

,person_id,transcript
4990,67318,I was hired by a woman with a lot of money. I ...


#**Step 4** หา “ผู้บงการ” จากคำให้การของฆาตกร

คำให้การของฆาตกร
"ผมถูกผู้หญิงที่รวยมากจ้างมา ผมไม่รู้ชื่อเธอ แต่รู้ว่าเธอสูงประมาณ 5 ฟุต 5 นิ้ว (65") หรือ 5 ฟุต 7 นิ้ว (67") เธอผมสีแดง และเธอขับรถ Tesla Model S ผมรู้ว่าเธอไปงาน SQL Symphony Concert 3 ครั้ง ในเดือนธันวาคม 2017"

สรุปเบาะแสผู้บงการ:

* เพศหญิง, ผมสีแดง

* สูง ~65–67 นิ้ว (≈165–170 ซม.)

* ขับ Tesla Model S

* เช็กอินงาน “SQL Symphony Concert” ≥ 3 ครั้งใน Dec-2017

In [54]:
# 1) คัดจากใบขับขี่ตามรูปร่าง/รถ
dl_filt = dl.query(
    "hair_color.str.lower()=='red' and car_make=='Tesla' and car_model=='Model S' and height>=65 and height<=67",
    engine='python'
)

# 2) ผูกกับ person เพื่อได้ person_id / name
cand = (dl_filt
        .merge(person[['id','name','license_id']], left_on='id', right_on='license_id', how='inner')
        .rename(columns={'id_x':'license_row_id','id_y':'person_id'}))

cand[['person_id','name','hair_color','height','car_make','car_model']]

# 3) คนที่ไปงาน SQL Symphony Concert ≥ 3 ครั้ง ใน Dec 2017
# Convert date to datetime objects for correct filtering
fb['date_dt'] = pd.to_datetime(fb['date'], format='%Y%m%d', errors='coerce')

# If column name is event_name:
evt_col = 'event_name' if 'event_name' in fb.columns else (
          'event' if 'event' in fb.columns else fb.columns[1])

fb_sql = fb[
    (fb['date_dt'].dt.year == 2017) &
    (fb['date_dt'].dt.month == 12) &
    fb[evt_col].str.contains('SQL Symphony Concert', case=False, na=False)
]

print("fb_sql (SQL Symphony Concert in Dec 2017):")
display(fb_sql)

count_sql = fb_sql.groupby('person_id').size().reset_index(name='n_sql_visits')

print("\ncount_sql (Visit counts for SQL Symphony Concert in Dec 2017):")
display(count_sql)


# 4) ตัดให้เหลือผู้บงการ
master = (cand.merge(count_sql, on='person_id', how='left')
               .fillna({'n_sql_visits':0})
               .query('n_sql_visits >= 3'))

master[['person_id','name','n_sql_visits','hair_color','height','car_make','car_model']]

fb_sql (SQL Symphony Concert in Dec 2017):


,person_id,event_id,event_name,date,date_dt
4087,62596,1143,SQL Symphony Concert,20171225,2017-12-25
4392,19260,1143,SQL Symphony Concert,20171214,2017-12-14
6552,58898,1143,SQL Symphony Concert,20171220,2017-12-20
6620,69699,1143,SQL Symphony Concert,20171214,2017-12-14
6717,19292,1143,SQL Symphony Concert,20171213,2017-12-13
7266,43366,1143,SQL Symphony Concert,20171207,2017-12-07
8395,92343,1143,SQL Symphony Concert,20171212,2017-12-12
10909,28582,1143,SQL Symphony Concert,20171220,2017-12-20
10910,28582,1143,SQL Symphony Concert,20171215,2017-12-15
13719,81526,1143,SQL Symphony Concert,20171202,2017-12-02



count_sql (Visit counts for SQL Symphony Concert in Dec 2017):


,person_id,n_sql_visits
0,11173,1
1,19260,1
2,19292,1
3,24397,1
4,24556,3
5,28582,2
6,43366,1
7,58898,1
8,62596,1
9,67318,1


,person_id,name,n_sql_visits,hair_color,height,car_make,car_model
0,99716,Miranda Priestly,3.0,red,66,Tesla,Model S


In [60]:
killer_name = killer_name            # จากขั้นก่อนหน้า
mastermind_name = master['name'].iloc[0]
print('Killer  :', killer_name)
print('Boss    :', mastermind_name)


Killer  : Jeremy Bowers
Boss    : Miranda Priestly


# Killer  : Jeremy Bowers
#Boss    : Miranda Priestly

# สรุปขั้นตอนการสืบคดี
เริ่มจาก crime_scene_report.csv กรองคดี murder 2018-01-15 ที่ Pandas City แล้วอ่านคำบรรยายซึ่งชี้ไปหาพยาน 2 คน (บ้านสุดถนน Northwestern Dr และ Annabel บน Franklin Ave). จากนั้นค้นพยานใน person.csv แล้วเปิดคำให้การใน interview.csv ได้เบาะแสว่า คนร้ายถือกระเป๋า Get Fit Now เลขสมาชิกขึ้นต้น “48Z”, เป็น gold member, ป้ายทะเบียนมี “H42W”, และไปยิมวันที่ 2018-01-09. ฉันจึงคัดสมาชิกจาก get_fit_now_member.csv (48Z+gold) เชื่อม get_fit_now_check_in.csv (เช็คอิน 2018-01-09) แล้วแม็ปกับ person.csv และ drivers_license.csv เพื่อตัดด้วยป้ายทะเบียน H42W จนได้ Jeremy Bowers เป็นฆาตกร. ต่อมาอ่านคำให้การของเขา พบว่าผู้จ้างเป็นผู้หญิงผมแดง สูงราว 65–67 นิ้ว ขับ Tesla Model S และไป SQL Symphony Concert อย่างน้อย 3 ครั้งใน Dec-2017 จึงกรองจาก drivers_license.csv และยืนยันด้วย facebook_event_checkin.csv จบที่ผู้บงการคือ Miranda Priestly.